In [1]:
import sys, datetime
sys.path.append("../../scripts/")
from s3_support import *

In [5]:
print("drop old values")
q = '''drop table if exists donors_orgs'''
redshift_query_write(q, schema='public')

drop old values


In [9]:
print("calculate data")
q = '''select
            org,
            email as donor,
            min(date) as starting_date,
            max(extract('epoch' from date))::int - min(extract('epoch' from date))::int as activity_duration,
            count(id) as transactions,
            sum(amount) as volume,
            count(distinct(t.recurring)) as recurring_origin,
            count(distinct(case when recurring!=0 then recurring else null end)) as recurring,
            sum(purchases_quantity) as purchases,
            sum(donations_count) as donations,
            sum(events_count) as events,
            sum(registrations_count) as registrations,
            sum(auctiondonation_count) as auctiondonations,
            sum(auctionpurchase_count) as auctionpurchases,
            sum(gift_assist_count) as giftassist,
            sum(matchinggifts_count) as matchinggifts
        into public.donors_orgs
        from production.transactions
        where
            status='A'
        group by org, email'''
redshift_query_write(q, schema='production')

calculate data


In [10]:
q = '''select * from public.donors_orgs limit 5'''
redshift_query_read(q, schema='public')

,org,donor,activity_duration,transactions,volume,recurring,purchases,donations,events,registrations,auctiondonations,auctionpurchases,giftassist,matchinggifts
0,6,eff036d4823c8620b3fc4fad2ccd2903ceb77991,130636800,96,14337.31,1,0,5,182,0,0,0,88,0
1,25,a455e35b4d05416aedc84e4020dc907a52afcc09,0,1,1000.00,0,0,0,0,0,0,0,0,0
2,39,85e64ac590ce5296ab09532f94f65489c0005584,0,1,1000.00,0,0,1,0,0,0,0,0,0
3,42,d0ac1f5754f05927a41ca1c75d1f983dccb9a25a,0,1,100.00,0,0,0,0,0,0,0,0,0
4,42,7ff85c67fb6d2cbc28244c99ee10a5bf7a9e0937,0,1,100.00,0,0,0,1,0,0,0,0,0
